<a href="https://colab.research.google.com/github/AliAkbarBadri/complex-networks-basics/blob/main/influence_maximization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setuping

In [22]:
import pandas as pd
import networkx as nx
import scipy.io
import matplotlib.pyplot as plt
import collections
import numpy as np
from collections import Counter
import sys
%matplotlib inline
import matplotlib.pyplot as plt
from random import uniform, seed
import time
import pickle
import math

In [2]:
! wget "https://github.com/AliAkbarBadri/complex-networks-basics/blob/main/facebook101_princton_weighted.mat?raw=true" -O "facebook101_princton_weighted.mat"

--2020-12-08 18:33:06--  https://github.com/AliAkbarBadri/complex-networks-basics/blob/main/facebook101_princton_weighted.mat?raw=true
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/AliAkbarBadri/complex-networks-basics/raw/main/facebook101_princton_weighted.mat [following]
--2020-12-08 18:33:07--  https://github.com/AliAkbarBadri/complex-networks-basics/raw/main/facebook101_princton_weighted.mat
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AliAkbarBadri/complex-networks-basics/main/facebook101_princton_weighted.mat [following]
--2020-12-08 18:33:07--  https://raw.githubusercontent.com/AliAkbarBadri/complex-networks-basics/main/facebook101_princton_weighted.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 

In [ ]:
! wget "https://github.com/AliAkbarBadri/complex-networks-basics/blob/main/marg_gain2.pickle?raw=true" -O marg_gain.pickle

--2020-12-03 23:37:01--  https://github.com/AliAkbarBadri/complex-networks-basics/blob/main/marg_gain.pickle?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/AliAkbarBadri/complex-networks-basics/raw/main/marg_gain.pickle [following]
--2020-12-03 23:37:01--  https://github.com/AliAkbarBadri/complex-networks-basics/raw/main/marg_gain.pickle
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AliAkbarBadri/complex-networks-basics/main/marg_gain.pickle [following]
--2020-12-03 23:37:01--  https://raw.githubusercontent.com/AliAkbarBadri/complex-networks-basics/main/marg_gain.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.git

# Reading Data

In [3]:
mat = scipy.io.loadmat('facebook101_princton_weighted.mat')
print(mat.keys())
print(mat['A'].shape[0])
print(mat['A'])

dict_keys(['__header__', '__version__', '__globals__', 'A'])
6596
  (41, 0)	0.23341986187511532
  (74, 0)	-0.8310836341707424
  (183, 0)	0.3630774693417993
  (199, 0)	-0.4255819109410841
  (330, 0)	-0.8915522179046713
  (342, 0)	0.6004699460747422
  (438, 0)	0.559033155005757
  (638, 0)	-0.15789912487319158
  (639, 0)	-0.3689795200377264
  (700, 0)	0.667096028355807
  (762, 0)	0.6388290129260563
  (790, 0)	-0.5971903653879409
  (813, 0)	-0.9203448232766431
  (908, 0)	-0.5661217215565775
  (1001, 0)	-0.40887388509123834
  (1074, 0)	0.14383939484847108
  (1084, 0)	0.4229554254707699
  (1179, 0)	0.3118105095583956
  (1195, 0)	0.014127530389211218
  (1226, 0)	-0.47309716640306787
  (1252, 0)	-0.013995007645000168
  (1343, 0)	-0.3969440595287481
  (1415, 0)	0.49268599971253546
  (1435, 0)	-0.21118357393040132
  (1489, 0)	-0.250254338391553
  :	:
  (3083, 6595)	-0.2566179533318984
  (3255, 6595)	-0.16483967626475793
  (3311, 6595)	-0.9284036578868178
  (3487, 6595)	0.9428205382791961
  (3647

In [4]:
G = nx.from_scipy_sparse_matrix(mat['A'])
print(G.number_of_nodes())
print(G.number_of_edges())

6596
293320


In [5]:
G.get_edge_data(6568,6595)

{'weight': -0.05460279445991589}

# Influence Maximization
https://www.cs.cornell.edu/home/kleinber/kdd03-inf.pdf 

https://www.cs.cmu.edu/~jure/pubs/detect-kdd07.pdf

In [ ]:
def IC(g,S,p=0.5,mc=1000):
  spread = []
  for i in range(mc):
    new_active, A = S[:], S[:]
    while new_active:
      new_ones = []
      for node in new_active:
          np.random.seed(i)
          success = np.random.uniform(0,1,len(g[node])) < p
          new_ones += list(np.extract(success, list(G[node])))
      new_active = list(set(new_ones) - set(A))
      A += new_active
    spread.append(len(A))
  return(np.mean(spread))

## Greedy

In [ ]:
g=G
k=10
p=0.1
mc=5
S, spread  = [], []
for i in range(k):
  print("K: ",i)
  best_spread = 0
  for j in set(range(g.number_of_nodes()))-set(S):
    if j % 250==0:
      print(" Node:",j)
    s = IC(g,S + [j],p,mc)
    if s > best_spread:
      best_spread, node = s, j
  S.append(node)
  spread.append(best_spread)
S,spread

K: 0
 Node: 0
 Node: 250
 Node: 500
 Node: 750
 Node: 1000
 Node: 1250
 Node: 1500
 Node: 1750
 Node: 2000
 Node: 2250
 Node: 2500
 Node: 2750
 Node: 3000
 Node: 3250
 Node: 3500
 Node: 3750
 Node: 4000
 Node: 4250
 Node: 4500
 Node: 4750
 Node: 5000
 Node: 5250
 Node: 5500
 Node: 5750
 Node: 6000
 Node: 6250
 Node: 6500
K: 1
 Node: 0
 Node: 250
 Node: 500
 Node: 750
 Node: 1000
 Node: 1250
 Node: 1500
 Node: 1750
 Node: 2000
 Node: 2250
 Node: 2500
 Node: 2750
 Node: 3000
 Node: 3250
 Node: 3500
 Node: 3750
 Node: 4000
 Node: 4250
 Node: 4500
 Node: 4750
 Node: 5000
 Node: 5250
 Node: 5500
 Node: 5750
 Node: 6000
 Node: 6250
 Node: 6500
K: 2
 Node: 0
 Node: 250
 Node: 500
 Node: 750
 Node: 1000
 Node: 1250
 Node: 1500
 Node: 1750
 Node: 2000
 Node: 2250
 Node: 2500
 Node: 2750
 Node: 3000
 Node: 3250
 Node: 3500
 Node: 3750
 Node: 4000
 Node: 4250
 Node: 4500
 Node: 4750
 Node: 5000
 Node: 5250
 Node: 5500
 Node: 5750
 Node: 6000
 Node: 6250
 Node: 6500
K: 3
 Node: 0
 Node: 250
 Node:

## Lazy Greedy

In [ ]:
g=G
k=10
p=0.1
mc=5

start_time = time.time() 
marg_gain = []
for i,node in enumerate(range(g.number_of_nodes())):
  if i % 250 == 0:
    print("node: ",i)
  marg_gain.append(IC(g,[node],p,mc))

node:  0
node:  250
node:  500
node:  750
node:  1000
node:  1250
node:  1500
node:  1750
node:  2000
node:  2250
node:  2500
node:  2750
node:  3000
node:  3250
node:  3500
node:  3750
node:  4000
node:  4250
node:  4500
node:  4750
node:  5000
node:  5250
node:  5500
node:  5750
node:  6000
node:  6250
node:  6500


In [ ]:
# pickle_out = open("marg_gain.pickle","wb")
# pickle.dump(marg_gain, pickle_out)
# pickle_out.close()

In [ ]:
# pickle_in = open("marg_gain.pickle","rb")
# marg_gain = pickle.load(pickle_in)

In [ ]:
Q = sorted(zip(range(g.number_of_nodes()),marg_gain), key=lambda x: x[1],reverse=True)
S, spread, SPREAD = [Q[0][0]], Q[0][1], [Q[0][1]]

In [ ]:
Q, LOOKUPS, timelapse = Q[1:], [g.number_of_nodes()], [time.time()-start_time]

In [ ]:
for i in range(k-1):
  print("K:",i)
  check, node_lookup = False, 0
  while not check:
    if node_lookup%100==0:
      print(" node_lookup:", node_lookup)
    node_lookup += 1        
    current = Q[0][0]
    Q[0] = (current, IC(g,S+[current],p,mc) - spread)
    Q = sorted(Q, key = lambda x: x[1], reverse = True)
    check = (Q[0][0] == current)
  spread += Q[0][1]
  S.append(Q[0][0])
  SPREAD.append(spread)
  LOOKUPS.append(node_lookup)
  timelapse.append(time.time() - start_time)
  Q = Q[1:]
S,SPREAD,timelapse,LOOKUPS

K: 0
 node_lookup: 0
 node_lookup: 100
 node_lookup: 200
 node_lookup: 300
 node_lookup: 400
 node_lookup: 500
 node_lookup: 600
 node_lookup: 700
 node_lookup: 800
 node_lookup: 900
 node_lookup: 1000
 node_lookup: 1100
 node_lookup: 1200
 node_lookup: 1300
 node_lookup: 1400
 node_lookup: 1500
 node_lookup: 1600
 node_lookup: 1700
 node_lookup: 1800
 node_lookup: 1900
 node_lookup: 2000
 node_lookup: 2100
 node_lookup: 2200
 node_lookup: 2300
 node_lookup: 2400
 node_lookup: 2500
 node_lookup: 2600
 node_lookup: 2700
 node_lookup: 2800
 node_lookup: 2900
 node_lookup: 3000
 node_lookup: 3100
 node_lookup: 3200
 node_lookup: 3300
 node_lookup: 3400
 node_lookup: 3500
 node_lookup: 3600
 node_lookup: 3700
 node_lookup: 3800
 node_lookup: 3900
 node_lookup: 4000
 node_lookup: 4100
 node_lookup: 4200
 node_lookup: 4300
 node_lookup: 4400
 node_lookup: 4500
 node_lookup: 4600
 node_lookup: 4700
 node_lookup: 4800
 node_lookup: 4900
 node_lookup: 5000
 node_lookup: 5100
 node_lookup: 5200


([21, 4138, 41, 5, 1877, 6363, 4773, 4011, 2707, 2526],
 [5783.6,
  5786.2,
  5788.4,
  5790.2,
  5791.8,
  5793.2,
  5794.6,
  5796.0,
  5797.4,
  5798.8],
 [8014.99306511879,
  16358.044126749039,
  16360.450620412827,
  16370.002128601074,
  16371.261121034622,
  16385.71291732788,
  16386.936264038086,
  16388.16443991661,
  16390.605731248856,
  16391.82336997986],
 [6596, 6132, 2, 8, 1, 12, 1, 1, 2, 1])

# Outbreak Detection

In [85]:
sampled_graph = G.subgraph(list(range(0,1000)))

In [70]:
def IC(g,S,p=0.5,mc=1000):
  spread = []
  cost = []
  for i in range(mc):
    new_active, A = S[:], S[:]
    while new_active:
      new_ones = []
      for node in new_active:
        np.random.seed(i)
        success = np.random.uniform(0,1,len(g[node])) < p
        new_ones += list(np.extract(success, list(G[node]))) 
      new_active = list(set(new_ones) - set(A))
      A += new_active
    spread.append(len(A))
    
  cost = np.sum([np.sum([abs(G.get_edge_data(node_pair[0],node_pair[1])['weight']) for node_pair in G.edges(s)]) for s in S])
  return(np.mean(spread), cost)

## Lazy Greedy

In [86]:
sampled_graph.number_of_nodes(), sampled_graph.number_of_edges()

(1000, 6935)

In [97]:
start_time = time.time() 
g=sampled_graph
p=0.1
mc=5


reward_cost = []
start_time = time.time()
for i,node in enumerate(range(g.number_of_nodes())):
  if i % 100 == 0:
    print("node: ",i)
  reward_cost.append(IC(g,[node],p,mc))
print(time.time() - start_time)

node:  0
node:  100
node:  200
node:  300
node:  400
node:  500
node:  600
node:  700
node:  800
node:  900
242.68867993354797


In [98]:
reward_cost[:10]

[(434.4, 49.60587559216012),
 (103.4, 6.132779829633405),
 (103.4, 6.031329452008562),
 (1.8, 8.450456246905508),
 (242.4, 21.936088834377628),
 (516.2, 249.3511263984574),
 (417.4, 74.1256862419249),
 (511.2, 64.87171620953266),
 (338.6, 27.51054819849458),
 (242.4, 27.217282789194297)]

In [99]:
marg_gain = [0.3*item[0]-item[1] for item in reward_cost]
marg_gain[:10]

[80.71412440783988,
 24.887220170366597,
 24.98867054799144,
 -7.910456246905508,
 50.78391116562237,
 -94.4911263984574,
 51.09431375807509,
 88.48828379046732,
 74.06945180150542,
 45.5027172108057]

In [115]:
Q = sorted(zip(range(g.number_of_nodes()),marg_gain), key=lambda x: x[1],reverse=True)
S, spread, SPREAD = [Q[0][0]], Q[0][1], [Q[0][1]]

In [116]:
Q, LOOKUPS, timelapse = Q[1:], [g.number_of_nodes()], [time.time()-start_time]

In [117]:
Q[:10]

[(931, 126.46680528282307),
 (885, 124.37249705189882),
 (701, 118.36678850962114),
 (362, 117.70600999275291),
 (652, 116.96473790454732),
 (655, 114.05351878323165),
 (418, 113.49409045604823),
 (798, 113.2165233608207),
 (585, 112.71140844623807),
 (123, 112.53379569514874)]

In [118]:
prev_spred = -1
sum = 1
i = 0
while True:
  print("K:",i)
  check, node_lookup = False, 0
  while not check:
    if node_lookup%100==0:
      print(" node_lookup:", node_lookup)
    node_lookup += 1        
    current = Q[0][0]
    reward_cost_temp = IC(g,S+[current],p,mc)
    Q[0] = (current, (0.3*reward_cost_temp[0]-reward_cost_temp[1]) - spread)
    Q = sorted(Q, key = lambda x: x[1], reverse = True)
    check = (Q[0][0] == current)
  if spread + Q[0][1] >= spread:
    spread += Q[0][1]
    S.append(Q[0][0])
    SPREAD.append(spread)
    LOOKUPS.append(node_lookup)
    timelapse.append(time.time() - start_time)
    Q = Q[1:]
    i+=1
  else:
    print("Bad one, index:",Q[0][0],', Value:', Q[0][1])
    break
S,SPREAD,timelapse,LOOKUPS

K: 0
 node_lookup: 0
 node_lookup: 100
 node_lookup: 200
 node_lookup: 300
 node_lookup: 400
 node_lookup: 500
 node_lookup: 600
 node_lookup: 700
 node_lookup: 800
K: 1
 node_lookup: 0
K: 2
 node_lookup: 0
K: 3
 node_lookup: 0
K: 4
 node_lookup: 0
K: 5
 node_lookup: 0
K: 6
 node_lookup: 0
K: 7
 node_lookup: 0
K: 8
 node_lookup: 0
K: 9
 node_lookup: 0
K: 10
 node_lookup: 0
K: 11
 node_lookup: 0
K: 12
 node_lookup: 0
K: 13
 node_lookup: 0
Bad one, index: 231 , Value: -0.10157343103034577


([922, 556, 675, 624, 126, 357, 754, 980, 369, 890, 478, 664, 390, 755],
 [127.3596441398733,
  127.65773371564205,
  127.94262768678156,
  128.22607072717022,
  128.4126953629749,
  128.571072739883,
  128.71923210248107,
  128.85331586455482,
  128.98104041780482,
  129.08737267662838,
  129.14598229973672,
  129.18127880297243,
  129.206379895828,
  129.22471102136547],
 [3261.5113637447357,
  3585.4916903972626,
  3585.941938638687,
  3586.532333612442,
  3587.1196587085724,
  3587.7037048339844,
  3588.254677295685,
  3589.4248790740967,
  3589.934312105179,
  3590.4335236549377,
  3591.0248720645905,
  3591.6091527938843,
  3592.2003362178802,
  3592.790076494217],
 [1000, 835, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1])